In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="0"
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import os
%load_ext autoreload
%autoreload 2
DATA_DIR='data_nuclis'
DATA_DIR_DEV='data_nuclis_dev'
#test_file='2tg_E98p_211a001{0}.png'
from image_utils import *


object_detection.utils not imported


In [33]:
generate_images_folder(DATA_DIR)
generate_images_folder(DATA_DIR_DEV)

object_detection.utils not imported
folder data_nuclis/2tg_E103p_223a001
folder data_nuclis_dev/2tg_E98p_224a001


In [37]:
get_py_px('cl1_2tg_E103p_223a001_00000_12_0_0_0.png')

object_detection.utils not imported


['12', '0']

In [21]:
generate_images_folder_v2(DATA_DIR)
#generate_images_folder_v2(DATA_DIR_DEV)

df data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001{0}.png
folder data_nuclis/2tg_E103p_223a001 0 0
folder data_nuclis/2tg_E103p_223a001 0 1
folder data_nuclis/2tg_E103p_223a001 1 0
folder data_nuclis/2tg_E103p_223a001 1 1
df data_nuclis/2tg_E98p_224a001/2tg_E98p_224a001{0}.png
folder data_nuclis/2tg_E98p_224a001 0 0
folder data_nuclis/2tg_E98p_224a001 0 1
folder data_nuclis/2tg_E98p_224a001 1 0
folder data_nuclis/2tg_E98p_224a001 1 1


In [3]:
X_train,Y_train,file_names_train,PY_train,PX_train=get_images_data(DATA_DIR)
X_dev,Y_dev,file_names_dev,PY_dev,PX_dev=get_images_data(DATA_DIR_DEV)

In [43]:
PY_dev.shape

(1, 7424)

In [1]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'

nx=X_train.shape[0]
num_classes=1
X = tf.placeholder(tf.float32, [nx,None],name="X")
W = tf.Variable(tf.zeros([num_classes, nx]),name="W")
b = tf.Variable(tf.zeros([num_classes,1]),name="b")
Z = tf.matmul(W, X) + b
Ypred=tf.sigmoid(Z)
Y = tf.placeholder(tf.float32, [num_classes, None])
cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z),  labels = tf.transpose(Y)))#+tf.nn.l2_loss(W)
train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(100):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train})
        print i,"train",j,a
        print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev})
    Ydev=sess.run(Ypred,feed_dict={X:patches_dev})
    Ytrain=sess.run(Ypred,feed_dict={X:patches_train})
    draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],10))
    draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],10))

NameError: name 'X_train' is not defined

In [3]:
import tensorflow as tf
DEV_FILE='data_nuclis_dev/2tg_E98p_224a001/2tg_E98p_224a001.png'
TRAIN_FILE='data_nuclis/2tg_E103p_223a001/2tg_E103p_223a001.png'
NUM_EPOCHS=3000

nx=X_train.shape[0]
num_classes=1
num_hidden1=4000
num_hidden2=500
with tf.variable_scope('test02') as scope:
    X = tf.placeholder(tf.float32, [nx,None],name="X")
    keep_prob = tf.placeholder(tf.float32)
    W1 = tf.get_variable("W1", [num_hidden1,nx], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b1 = tf.get_variable("b1",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden1,1)),dtype=tf.float32))
    W2 = tf.get_variable("W2", [num_hidden2,num_hidden1], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b2 = tf.get_variable("b2",dtype=tf.float32,initializer=tf.constant(np.zeros((num_hidden2,1)),dtype=tf.float32))
    W3 = tf.get_variable("W3", [num_classes,num_hidden2], initializer = tf.contrib.layers.xavier_initializer(seed = 1),dtype=tf.float32)
    b3 = tf.get_variable("b3",dtype=tf.float32,initializer=tf.constant(np.zeros((num_classes,1)),dtype=tf.float32))
    Z1 = tf.matmul(W1, X) + b1
    A1=tf.nn.dropout(tf.nn.relu(Z1),keep_prob)
    Z2 = tf.matmul(W2, A1) + b2
    A2=tf.nn.dropout(tf.nn.relu(Z2),keep_prob)
    Z3 = tf.matmul(W3,A2)+b3
    Ypred=tf.sigmoid(Z3)
    Y = tf.placeholder(tf.float32, [num_classes, None])
    regul=tf.nn.l2_loss(W1)+tf.nn.l2_loss(W2)+tf.nn.l2_loss(W3)
    cost=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = tf.transpose(Z3),  labels = tf.transpose(Y)))+0.003*regul
    #train_step = tf.train.GradientDescentOptimizer(0.001).minimize(cost)
    train_step = tf.train.AdamOptimizer(0.00001).minimize(cost)
    correct_prediction = tf.equal(Y, tf.cast(tf.greater(Ypred,0.5),"float"))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
patches_dev,slices_dev=patches_of_image(DEV_FILE,40,40)
patches_train,slices_train=patches_of_image(TRAIN_FILE,40,40)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(NUM_EPOCHS):
        _,j,a=sess.run([train_step,cost,accuracy],feed_dict={X:X_train,Y:Y_train,keep_prob:0.5})
        if i%100==0:
            print i,"train",j,a
            print i,"test",sess.run([cost,accuracy],feed_dict={X:X_dev,Y:Y_dev,keep_prob:1.})
            Ydev=sess.run(Ypred,feed_dict={X:patches_dev,keep_prob:1.})
            Ytrain=sess.run(Ypred,feed_dict={X:patches_train,keep_prob:1.})
            draw_rectangles(DEV_FILE,filter_rectangles(slices_dev[Ydev>=0.5],30))
            draw_rectangles(TRAIN_FILE,filter_rectangles(slices_train[Ytrain>=0.5],20))
        #if i==2000:
        #    print "Canviant train_step"
        #    train_step=tf.train.GradientDescentOptimizer(0.0001).minimize(cost)

0 train 8.86087 0.489089
0 test [8.5396442, 0.6982758]
100 train 7.73098 0.910271
100 test [7.7137432, 0.90660912]
200 train 7.23465 0.942814
200 test [7.3099766, 0.90708804]
300 train 6.82512 0.946012
300 test [6.9223347, 0.90996158]
400 train 6.44129 0.948269
400 test [6.556386, 0.90660906]
500 train 6.07961 0.951467
500 test [6.2025352, 0.91187727]
600 train 5.73843 0.956922
600 test [5.8697162, 0.91618764]
700 train 5.41629 0.959932
700 test [5.5567245, 0.92073745]
800 train 5.11234 0.965199
800 test [5.262063, 0.9243294]
900 train 4.82514 0.970843
900 test [4.9917488, 0.92768186]
1000 train 4.557 0.975545
1000 test [4.7285337, 0.93199223]
1100 train 4.30599 0.975922
1100 test [4.4800267, 0.9363026]
1200 train 4.07018 0.978179
1200 test [4.2670331, 0.93103439]
1300 train 3.84625 0.983634
1300 test [4.042295, 0.93582368]
1400 train 3.63626 0.983634
1400 test [3.844888, 0.9370209]
1500 train 3.4418 0.985892
1500 test [3.6526983, 0.93941557]
1600 train 3.25677 0.987396
1600 test [3.46

In [13]:
s=slice(3,5)
range(10)[s]

[3, 4]

In [ ]:
from train import train3
train3(X_train.T,Y_train.T,PY_train.T,PX_train.T,X_dev.T,Y_dev.T,PY_dev.T,PX_dev.T,optimizer=tf.train.AdamOptimizer(0.0001))

0 train 23.9162 0.5725 78.202 99.9154
0 test [5.3932381, 0.7025001, 13.922648, 16.790409]
0 train 15.5374 0.511 49.106 51.8721
0 test [3.2675374, 0.72500002, 14.10957, 12.970984]
0 train 13.4475 0.5165 43.132 48.7512
0 test [5.7725801, 0.29300001, 14.295765, 22.374033]
0 train 17.2969 0.4975 52.6165 72.1675
0 test [7.5116644, 0.32303369, 19.21022, 26.143927]
10 train 5.46746 0.731 20.6362 23.3415
10 test [2.9367003, 0.83050001, 12.68915, 11.909973]
10 train 5.42044 0.742 21.6828 22.9075
10 test [3.0129054, 0.82849997, 13.209654, 11.316304]
10 train 5.11409 0.744 18.8966 22.7553
10 test [2.8397169, 0.81850004, 11.820564, 11.086817]
10 train 5.70109 0.746 23.4665 23.8075
10 test [3.0181255, 0.83356744, 13.425144, 11.878214]
20 train 4.22969 0.8225 18.394 18.5201
20 test [2.8868718, 0.84500009, 12.761957, 11.708898]
20 train 4.27465 0.82 18.62 18.7589
20 test [2.9635694, 0.83399999, 13.2512, 11.21452]
20 train 4.02581 0.829 16.6582 18.3885
20 test [2.7704153, 0.83400011, 11.842934, 10.905

220 train 2.36451 0.9475 10.3093 12.0721
220 test [2.8193028, 0.93650007, 14.167626, 12.344589]
220 train 2.36407 0.9535 10.4891 11.9711
220 test [2.9039788, 0.92950004, 14.970996, 11.910748]
220 train 2.27561 0.951 9.78784 11.7126
220 test [2.7725317, 0.92950004, 14.01751, 11.683309]
220 train 2.72631 0.9485 12.0656 14.017
220 test [3.1044834, 0.9367978, 15.7734, 13.29567]
230 train 2.29883 0.945 10.2698 11.3993
230 test [2.8443794, 0.93949997, 14.34376, 12.409377]
230 train 2.32374 0.956 10.4214 11.6693
230 test [2.9277163, 0.93050003, 15.093382, 11.9709]
230 train 2.22952 0.959 9.67072 11.5225
230 test [2.7957997, 0.93199998, 14.16533, 11.726377]
230 train 2.61953 0.959 11.5843 13.4813
230 test [3.1281333, 0.93960679, 15.922171, 13.40108]
240 train 2.21288 0.944 9.63041 11.2643
240 test [2.8685493, 0.93650007, 14.467112, 12.525514]
240 train 2.24579 0.955 10.0239 11.209
240 test [2.9555204, 0.93050003, 15.309155, 12.007408]
240 train 2.15803 0.958 9.50578 10.9708
240 test [2.8102739

In [7]:
X_train.shape

(4800, 9416)